# 🛡️ Simulação de Ataques e Troubleshooting SIMIR

## 📋 Visão Geral
Este notebook fornece um guia completo para:
- ✅ **Diagnosticar** por que o `notice.log` não está sendo gerado
- 🎯 **Simular ataques** reais de uma máquina Linux na mesma rede
- 🔍 **Verificar detecções** de port scan e força bruta
- 🛠️ **Solucionar problemas** comuns do sistema SIMIR

## ⚠️ Pré-requisitos
- Sistema SIMIR rodando (container Zeek ativo)
- Notebook Linux conectado na mesma rede
- Ferramentas: `nmap`, `hydra`, `ssh`, `curl`, `docker`
- Permissões sudo (para comandos Docker)

## 📖 Como Usar
1. Execute cada célula sequencialmente
2. Verifique os outputs para identificar problemas
3. Ajuste configurações conforme necessário
4. Execute simulações de ataque com responsabilidade

---

## 🔍 Seção 1: Verificação do Notice.log

### Problema Principal
O arquivo `notice.log` **não está sendo criado** ou está vazio. Isso geralmente indica:
- Scripts de detecção não carregados
- Falta de atividade que gere alertas
- Problemas de configuração do Zeek
- Erros de sintaxe nos scripts

Vamos diagnosticar sistematicamente!

In [ ]:
import subprocess
import os
import time
from datetime import datetime

def run_command(cmd, description=""):
    """Executa comando e exibe resultado formatado"""
    print(f"🔧 {description}")
    print(f"💻 Executando: {cmd}")
    print("─" * 50)
    
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=30)
        if result.stdout:
            print("✅ Output:")
            print(result.stdout)
        if result.stderr:
            print("⚠️ Error/Warning:")
            print(result.stderr)
        if result.returncode != 0:
            print(f"❌ Exit code: {result.returncode}")
        print("\n" + "="*70 + "\n")
        return result
    except subprocess.TimeoutExpired:
        print("⏰ Timeout: Comando demorou mais de 30 segundos")
        print("\n" + "="*70 + "\n")
        return None
    except Exception as e:
        print(f"❌ Erro: {e}")
        print("\n" + "="*70 + "\n")
        return None

# Verificar status do container Docker
run_command("sudo docker-compose ps", "Verificando status do container SIMIR")

# Verificar se o notice.log existe
run_command("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/logs/current/notice.log", 
           "Verificando se notice.log existe")

# Verificar outros logs para garantir que Zeek está funcionando
run_command("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/logs/current/", 
           "Listando todos os arquivos de log")

## 📜 Seção 2: Verificação dos Scripts Zeek

### Scripts Críticos para Detecção
O SIMIR depende de scripts personalizados para gerar alertas:
- `port-scan-detector.zeek` - Detecção de port scan
- `brute-force-detector.zeek` - Detecção de força bruta
- `local.zeek` - Configuração principal

Se estes não estiverem carregados, **nenhum alerta será gerado**!

In [ ]:
# Verificar quais scripts estão carregados
run_command("sudo docker exec SIMIR_Z grep -i 'port-scan\\|brute-force\\|local.zeek' /usr/local/zeek/logs/current/loaded_scripts.log", 
           "Verificando scripts de detecção carregados")

# Verificar status do Zeek
run_command("sudo docker exec SIMIR_Z zeekctl status", 
           "Verificando status do processo Zeek")

# Verificar se há erros no reporter.log
run_command("sudo docker exec SIMIR_Z cat /usr/local/zeek/logs/current/reporter.log", 
           "Verificando erros relatados pelo Zeek")

# Verificar se os scripts existem no diretório correto
run_command("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/share/zeek/site/", 
           "Verificando scripts no diretório site")

run_command("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/spool/installed-scripts-do-not-touch/site/", 
           "Verificando scripts instalados")

## 🔧 Seção 3: Diagnóstico do Container e Configuração

### Problemas Comuns
- Scripts não instalados após modificações
- Zeek não reiniciado após mudanças
- Problemas de interface de rede
- Configurações de Notice::policy incorretas

### Soluções Automáticas
Vamos tentar corrigir os problemas mais comuns automaticamente:

In [ ]:
# Função para corrigir problemas automaticamente
def fix_zeek_issues():
    print("🔧 TENTANDO CORRIGIR PROBLEMAS AUTOMATICAMENTE")
    print("="*60)
    
    # Reinstalar scripts (CRÍTICO!)
    run_command("sudo docker exec SIMIR_Z zeekctl install", 
               "Reinstalando scripts Zeek")
    
    # Reiniciar Zeek
    run_command("sudo docker exec SIMIR_Z zeekctl restart", 
               "Reiniciando Zeek")
    
    # Aguardar inicialização
    print("⏰ Aguardando 10 segundos para inicialização...")
    time.sleep(10)
    
    # Verificar status após correção
    run_command("sudo docker exec SIMIR_Z zeekctl status", 
               "Verificando status após correção")

# Executar correções
fix_zeek_issues()

# Verificar se notice.log foi criado agora
run_command("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/logs/current/notice.log", 
           "Verificando notice.log após correções")

## 🎯 Seção 4: Simulação de Port Scan

### Objetivo
Gerar tráfego de port scan para testar se o sistema de detecção está funcionando.

### ⚠️ IMPORTANTE - Configuração de Rede
Antes de executar, você precisa:
1. **Identificar o IP do servidor SIMIR** na sua rede
2. **Confirmar que está na mesma rede** que o SIMIR
3. **Usar apenas em ambiente de teste**

### Descoberta da Rede

In [ ]:
import socket
import ipaddress

def get_network_info():
    """Descobrir informações da rede local"""
    print("🌐 DESCOBRINDO INFORMAÇÕES DA REDE")
    print("="*50)
    
    # Descobrir IP local
    try:
        # Conectar a um IP externo para descobrir interface local
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        
        print(f"🖥️  IP Local: {local_ip}")
        
        # Calcular rede
        network = ipaddress.IPv4Network(f"{local_ip}/24", strict=False)
        print(f"🌍 Rede: {network}")
        
        return local_ip, str(network.network_address)
    except Exception as e:
        print(f"❌ Erro ao descobrir rede: {e}")
        return None, None

# Descobrir rede
local_ip, network_base = get_network_info()

# Verificar se nmap está instalado
run_command("which nmap", "Verificando se nmap está instalado")

if not local_ip:
    print("❌ Não foi possível descobrir a rede. Configure manualmente:")
    print("1. Execute: ip addr show")
    print("2. Identifique seu IP local")
    print("3. Configure a variável target_ip abaixo")
    target_ip = "192.168.1.100"  # CONFIGURE ESTE IP!
else:
    # Sugerir IP do SIMIR (geralmente o gateway ou um IP próximo)
    target_ip = input(f"🎯 Digite o IP do servidor SIMIR (sua rede é {network_base}/24): ")
    if not target_ip:
        # Usar IP base + 1 como padrão (gateway comum)
        target_ip = f"{network_base.rsplit('.', 1)[0]}.1"

print(f"🎯 Alvo definido: {target_ip}")
print(f"🖥️  Atacante: {local_ip}")

In [ ]:
# CONFIGURAÇÃO DO ALVO - AJUSTE CONFORME SUA REDE
TARGET_IP = "192.168.1.100"  # ⚠️ CONFIGURE ESTE IP PARA O SERVIDOR SIMIR!

print(f"🎯 INICIANDO SIMULAÇÃO DE PORT SCAN")
print(f"Alvo: {TARGET_IP}")
print("="*50)

# Verificar se o alvo está acessível
run_command(f"ping -c 3 {TARGET_IP}", f"Testando conectividade com {TARGET_IP}")

# 1. Port Scan SYN (rápido e discreto)
print("🔍 EXECUTANDO SYN SCAN...")
run_command(f"sudo nmap -sS -F {TARGET_IP}", 
           f"Port Scan SYN nas portas mais comuns")

# 2. Port Scan TCP Connect (mais detectável)
print("🔍 EXECUTANDO TCP CONNECT SCAN...")
run_command(f"nmap -sT -p 22,23,80,443,21,25,53,135,139,445,993,995,3389 {TARGET_IP}", 
           f"Port Scan TCP em portas críticas")

# 3. Port Scan UDP (para detectar serviços UDP)
print("🔍 EXECUTANDO UDP SCAN...")
run_command(f"sudo nmap -sU -p 53,67,68,123,161,514 {TARGET_IP}", 
           f"Port Scan UDP em portas comuns")

# 4. Scan intensivo (muitas portas para garantir detecção)
print("🔍 EXECUTANDO SCAN INTENSIVO...")
run_command(f"nmap -p 1-1000 {TARGET_IP}", 
           f"Port Scan intensivo (portas 1-1000)")

print("✅ SIMULAÇÃO DE PORT SCAN CONCLUÍDA!")
print("🔍 Agora vamos verificar se foi detectado...")

## 🔐 Seção 5: Simulação de Força Bruta

### Tipos de Ataques a Simular
1. **SSH Brute Force** - Múltiplas tentativas de login SSH
2. **FTP Brute Force** - Tentativas de autenticação FTP
3. **HTTP Brute Force** - Ataques a formulários web
4. **Força Bruta Genérica** - Conexões rejeitadas múltiplas

### ⚠️ Ética e Responsabilidade
- Use **APENAS** em ambientes de teste
- **NÃO** execute contra sistemas que não são seus
- Finalidade: **Testar o sistema de detecção SIMIR**

In [ ]:
import threading
import time

print(f"🔐 INICIANDO SIMULAÇÃO DE FORÇA BRUTA")
print(f"Alvo: {TARGET_IP}")
print("="*50)

# 1. SSH Brute Force Simulation
def ssh_bruteforce_simulation():
    """Simula tentativas de SSH brute force"""
    print("🔑 SIMULANDO SSH BRUTE FORCE...")
    
    # Lista de usuários comuns para testar
    usernames = ['admin', 'root', 'user', 'test', 'guest', 'ubuntu', 'centos']
    
    for i, username in enumerate(usernames, 1):
        cmd = f"ssh -o ConnectTimeout=3 -o StrictHostKeyChecking=no {username}@{TARGET_IP} 'exit' 2>/dev/null"
        print(f"  Tentativa {i}: {username}@{TARGET_IP}")
        subprocess.run(cmd, shell=True, capture_output=True)
        time.sleep(1)  # Pausa entre tentativas
    
    # Tentativas adicionais com usuários repetidos
    for i in range(5):
        cmd = f"ssh -o ConnectTimeout=2 -o StrictHostKeyChecking=no admin@{TARGET_IP} 'exit' 2>/dev/null"
        subprocess.run(cmd, shell=True, capture_output=True)
        time.sleep(0.5)

# 2. FTP Brute Force Simulation  
def ftp_bruteforce_simulation():
    """Simula tentativas de FTP brute force"""
    print("🗂️  SIMULANDO FTP BRUTE FORCE...")
    
    # Verificar se FTP está disponível
    cmd = f"nc -z -v {TARGET_IP} 21"
    result = subprocess.run(cmd, shell=True, capture_output=True)
    
    if result.returncode == 0:
        print("  ✅ Serviço FTP encontrado, simulando ataques...")
        # Usar curl para tentativas FTP
        for i in range(10):
            cmd = f"curl -u admin:password123 ftp://{TARGET_IP}/ --connect-timeout 3 2>/dev/null"
            subprocess.run(cmd, shell=True, capture_output=True)
            time.sleep(0.5)
    else:
        print("  ⚠️ Serviço FTP não encontrado, pulando...")

# 3. HTTP Brute Force Simulation
def http_bruteforce_simulation():
    """Simula tentativas de HTTP brute force"""
    print("🌐 SIMULANDO HTTP BRUTE FORCE...")
    
    # Tentar diferentes endpoints comuns
    endpoints = ['/admin', '/login', '/wp-admin', '/admin.php', '/login.php']
    
    for endpoint in endpoints:
        for i in range(5):
            # Simular POST requests de login
            cmd = f"curl -X POST -d 'username=admin&password=password{i}' http://{TARGET_IP}{endpoint} --connect-timeout 3 -s -o /dev/null"
            subprocess.run(cmd, shell=True, capture_output=True)
            
            # Simular GET requests com autenticação básica
            cmd = f"curl -u admin:password{i} http://{TARGET_IP}{endpoint} --connect-timeout 3 -s -o /dev/null"
            subprocess.run(cmd, shell=True, capture_output=True)
            time.sleep(0.3)

# 4. Força Bruta Genérica (conexões rejeitadas)
def generic_bruteforce_simulation():
    """Simula conexões múltiplas rejeitadas"""
    print("🔧 SIMULANDO FORÇA BRUTA GENÉRICA...")
    
    # Tentar conectar em portas fechadas múltiplas vezes
    closed_ports = [8080, 9090, 9999, 8888, 7777, 6666, 5555]
    
    for port in closed_ports:
        for i in range(3):
            cmd = f"nc -z -w 1 {TARGET_IP} {port} 2>/dev/null"
            subprocess.run(cmd, shell=True, capture_output=True)
            time.sleep(0.2)

# Executar simulações em paralelo para maior realismo
print("🚀 Executando simulações em paralelo...")

threads = [
    threading.Thread(target=ssh_bruteforce_simulation),
    threading.Thread(target=ftp_bruteforce_simulation),
    threading.Thread(target=http_bruteforce_simulation),
    threading.Thread(target=generic_bruteforce_simulation)
]

# Iniciar todas as threads
for thread in threads:
    thread.start()

# Aguardar conclusão
for thread in threads:
    thread.join()

print("✅ SIMULAÇÃO DE FORÇA BRUTA CONCLUÍDA!")
print("🔍 Aguardando alguns segundos para processamento pelo Zeek...")
time.sleep(5)

## 📊 Seção 6: Monitoramento da Detecção

### Verificação de Resultados
Agora vamos verificar se os ataques simulados foram detectados pelo SIMIR:

1. **Notice.log** - Arquivo principal de alertas
2. **Conn.log** - Registros de conexões para análise
3. **Alertas específicos** - Port scan e força bruta
4. **Estatísticas** - Contagem de detecções

In [ ]:
def check_detections():
    """Verifica se os ataques foram detectados"""
    print("🔍 VERIFICANDO DETECÇÕES")
    print("="*50)
    
    # 1. Verificar se notice.log foi criado
    result = run_command("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/logs/current/notice.log", 
                        "Verificando se notice.log existe agora")
    
    if "No such file" in result.stderr if result else True:
        print("❌ PROBLEMA: notice.log ainda não existe!")
        print("💡 Isso indica que os scripts não estão gerando alertas.")
        return False
    
    # 2. Verificar conteúdo do notice.log
    run_command("sudo docker exec SIMIR_Z cat /usr/local/zeek/logs/current/notice.log", 
               "Conteúdo completo do notice.log")
    
    # 3. Verificar alertas específicos de Port Scan
    run_command("sudo docker exec SIMIR_Z grep -i 'PortScan\\|Port_Scan' /usr/local/zeek/logs/current/notice.log", 
               "Alertas de Port Scan detectados")
    
    # 4. Verificar alertas específicos de Força Bruta
    run_command("sudo docker exec SIMIR_Z grep -i 'BruteForce\\|Brute_Force' /usr/local/zeek/logs/current/notice.log", 
               "Alertas de Força Bruta detectados")
    
    # 5. Contar total de alertas
    run_command("sudo docker exec SIMIR_Z wc -l /usr/local/zeek/logs/current/notice.log", 
               "Total de linhas no notice.log")
    
    # 6. Verificar conexões do nosso IP nos logs
    if local_ip:
        run_command(f"sudo docker exec SIMIR_Z grep '{local_ip}' /usr/local/zeek/logs/current/conn.log | head -10", 
                   f"Conexões do IP atacante ({local_ip}) no conn.log")
    
    # 7. Verificar conexões rejeitadas (S0, REJ)
    run_command("sudo docker exec SIMIR_Z grep -E 'S0|REJ' /usr/local/zeek/logs/current/conn.log | head -10", 
               "Conexões rejeitadas no conn.log")
    
    return True

# Executar verificações
detections_found = check_detections()

# 8. Monitoramento em tempo real (últimos 30 segundos)
print("\n🔴 MONITORAMENTO EM TEMPO REAL")
print("="*50)
print("Executando novos ataques e monitorando em tempo real...")

# Executar um novo scan rápido
def quick_scan():
    time.sleep(2)
    subprocess.run(f"nmap -sS -F {TARGET_IP} >/dev/null 2>&1", shell=True)

# Iniciar scan em background
scan_thread = threading.Thread(target=quick_scan)
scan_thread.start()

# Monitorar logs em tempo real por 15 segundos
run_command("timeout 15 sudo docker exec SIMIR_Z tail -f /usr/local/zeek/logs/current/notice.log", 
           "Monitoramento em tempo real do notice.log (15 segundos)")

scan_thread.join()

## 🛠️ Seção 7: Troubleshooting e Soluções

### Cenários Possíveis

#### ✅ **SUCESSO**: Notice.log criado com alertas
- Sistema funcionando perfeitamente
- Detecções de port scan e/ou força bruta encontradas
- Continue monitorando e ajustando thresholds conforme necessário

#### ⚠️ **PARCIAL**: Notice.log existe, mas sem alertas de port scan/força bruta
- Scripts carregados, mas thresholds muito altos
- Ataques não suficientemente intensos
- Necessário ajustar configurações

#### ❌ **PROBLEMA**: Notice.log não existe ou vazio
- Scripts não carregados corretamente
- Problemas de configuração do Zeek
- Notice::policy não configurado

### Soluções Automáticas

In [ ]:
def comprehensive_troubleshooting():
    """Troubleshooting completo e automático"""
    print("🔧 TROUBLESHOOTING COMPLETO")
    print("="*60)
    
    # 1. Verificar se scripts existem no diretório site
    print("\n1️⃣ VERIFICANDO SCRIPTS...")
    run_command("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/share/zeek/site/", 
               "Listando scripts no diretório site")
    
    # 2. Verificar conteúdo do local.zeek
    print("\n2️⃣ VERIFICANDO LOCAL.ZEEK...")
    run_command("sudo docker exec SIMIR_Z cat /usr/local/zeek/share/zeek/site/local.zeek", 
               "Conteúdo do local.zeek")
    
    # 3. Forçar reinstalação e restart
    print("\n3️⃣ FORÇANDO REINSTALAÇÃO...")
    run_command("sudo docker exec SIMIR_Z zeekctl stop", "Parando Zeek")
    run_command("sudo docker exec SIMIR_Z zeekctl install", "Reinstalando scripts")
    run_command("sudo docker exec SIMIR_Z zeekctl start", "Iniciando Zeek")
    
    time.sleep(10)  # Aguardar inicialização
    
    # 4. Verificar status após restart
    print("\n4️⃣ VERIFICANDO STATUS APÓS RESTART...")
    run_command("sudo docker exec SIMIR_Z zeekctl status", "Status do Zeek")
    
    # 5. Executar um teste simples de notice geração
    print("\n5️⃣ TESTE DIRETO DE NOTICE...")
    # Usar o script de teste de força bruta se existir
    run_command("test -f scripts/test-brute-force.sh && ./scripts/test-brute-force.sh || echo 'Script de teste não encontrado'", 
               "Executando teste de força bruta do SIMIR")
    
    # 6. Verificar novamente após teste
    time.sleep(5)
    run_command("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/logs/current/notice.log", 
               "Verificação final do notice.log")
    
    # 7. Se ainda não funcionar, mostrar configuração manual
    result = subprocess.run("sudo docker exec SIMIR_Z ls -la /usr/local/zeek/logs/current/notice.log", 
                           shell=True, capture_output=True, text=True)
    
    if "No such file" in result.stderr:
        print("\n❌ NOTICE.LOG AINDA NÃO EXISTE!")
        print("\n📋 SOLUÇÕES MANUAIS NECESSÁRIAS:")
        print("1. Verificar se os scripts personalizados estão no diretório correto")
        print("2. Verificar sintaxe dos scripts Zeek")
        print("3. Verificar configuração do Notice::policy")
        print("4. Verificar logs de erro detalhados")
        
        # Mostrar logs de erro detalhados
        run_command("sudo docker exec SIMIR_Z cat /usr/local/zeek/logs/current/stderr.log", 
                   "Erros no stderr.log")
        
        run_command("sudo docker exec SIMIR_Z zeekctl diag", 
                   "Diagnóstico completo do Zeek")
    
    else:
        print("\n✅ NOTICE.LOG EXISTE!")
        run_command("sudo docker exec SIMIR_Z cat /usr/local/zeek/logs/current/notice.log", 
                   "Conteúdo atual do notice.log")

# Executar troubleshooting completo
comprehensive_troubleshooting()

print("\n" + "="*70)
print("🎯 RESUMO FINAL")
print("="*70)
print("✅ Scripts verificados e reinstalados")
print("✅ Ataques de port scan simulados") 
print("✅ Ataques de força bruta simulados")
print("✅ Logs verificados e monitorados")
print("\n📊 PRÓXIMOS PASSOS:")
print("1. Verifique se há alertas no notice.log")
print("2. Ajuste thresholds se necessário")
print("3. Configure alertas por email")
print("4. Monitore o sistema continuamente")
print("\n🔗 Para mais informações, consulte o MANUAL_COMPLETO.md")
print("="*70)

## 🎯 Configuração de IP e Próximos Passos

### ⚠️ IMPORTANTE: Configurar IP do Alvo

**Antes de executar este notebook, você DEVE:**

1. **Descobrir o IP do servidor SIMIR:**
   ```bash
   # No servidor SIMIR, execute:
   ip addr show
   # ou
   hostname -I
   ```

2. **Editar a variável TARGET_IP** na Seção 4:
   ```python
   TARGET_IP = "SEU_IP_AQUI"  # Ex: 192.168.1.100
   ```

3. **Verificar conectividade:**
   ```bash
   ping IP_DO_SIMIR
   ```

### 🔧 Comandos Manuais Alternativos

Se preferir executar comandos manuais fora do notebook:

```bash
# 1. Verificar notice.log
sudo docker exec SIMIR_Z ls -la /usr/local/zeek/logs/current/notice.log

# 2. Reinstalar scripts
sudo docker exec SIMIR_Z zeekctl install
sudo docker exec SIMIR_Z zeekctl restart

# 3. Simular port scan
nmap -sS -F IP_DO_SIMIR

# 4. Verificar detecções
sudo docker exec SIMIR_Z tail -f /usr/local/zeek/logs/current/notice.log
```

### 📚 Documentação Adicional

- **Manual Completo**: `MANUAL_COMPLETO.md`
- **Detecção de Força Bruta**: `docs/brute-force-detection.md`
- **Scripts de Controle**: `./scripts/simir-control.sh`

---

**✅ SUCESSO**: Se este notebook ajudou a resolver o problema do notice.log!  
**❌ PROBLEMAS**: Consulte a seção de Troubleshooting no manual ou execute diagnósticos manuais.